In [74]:
%cd '/Users/yannickflores/Perso Project/foreshadownba/foreshadwownba-inference-pipeline'

/Users/yannickflores/Perso Project/foreshadownba/foreshadwownba-inference-pipeline


/Users/yannickflores/Library/Caches/pypoetry/virtualenvs/foreshadwownba-inference-pipeline-JnjMA73S-py3.11/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [75]:
import argparse

import joblib
import numpy as np
import pandas as pd

import yaml
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

from pipeline.utils.logs import get_logger


In [76]:
def rename_opponent_columns(training_df: pd.DataFrame) -> pd.DataFrame:
    """
    Rename opponent columns
    """
    training_df.columns = training_df.columns.str.replace("_y", "_opp")
    training_df.columns = training_df.columns.str.replace("_x", "")

    return training_df

In [77]:
# -----------------------------------------------
# Read input params

target_column = 'results'
group_cv_variable = 'id'
model = joblib.load("./models/model.joblib")


nba_games_inseason_dataset_final = pd.read_csv("./data/input/nba_games_inseason_dataset_final.csv")
columns_selected = pd.read_csv("./models/columns_selected.csv")
columns_selected.columns=['index', 'columns_name']


In [78]:
column_to_select = columns_selected['columns_name'].values

In [79]:
nba_games_inseason_dataset_final[nba_games_inseason_dataset_final['tm']=='ATL'].tail()

,id_season,id,game_nb,game_date,tm,opp,results,before_average_pts_tm,before_average_lastfivegame_pts_tm,before_average_lasttengame_pts_tm,...,before_ratio_win_ext,before_ratio_win_dom,before_streak_w_l,extdom_ext,last_game_overtime_Multiple_OT,last_game_overtime_NOT,last_game_overtime_OT,before_average_W_ratio_opp,before_average_pts_tm_opp,before_average_pts_opp_opp
13,2025,2024-11-18_ATL_SAC,15,2024-11-18,ATL,SAC,1,116.6,118.0,116.5,...,0.333333,0.500000,-1.0,1,0,1,0,0.571429,117.5,114.1
14,2025,2024-11-20_ATL_GSW,16,2024-11-20,ATL,GSW,0,116.1,115.6,115.4,...,0.428571,0.500000,1.0,1,0,1,0,0.769231,119.6,110.1
15,2025,2024-11-22_ATL_CHI,17,2024-11-22,ATL,CHI,0,114.9,112.4,113.6,...,0.375000,0.500000,-1.0,1,0,1,0,0.375000,115.4,122.9
16,2025,2024-11-25_DAL_ATL,18,2024-11-25,ATL,DAL,0,115.4,113.4,113.2,...,0.333333,0.500000,-2.0,0,0,1,0,0.529412,116.2,110.7
17,2025,2024-11-27_ATL_CLE,19,2024-11-27,ATL,CLE,0,115.6,111.4,115.8,...,0.333333,0.444444,-3.0,1,0,1,0,0.944444,123.4,111.1


In [80]:

list_columns_to_delete = [
    target_column,
    group_cv_variable,
    "id_season",
    "tm",
    "opp",
]

y_inseason = nba_games_inseason_dataset_final.loc[:, target_column].values
x_inseason = nba_games_inseason_dataset_final.drop(list_columns_to_delete, axis=1)
x_inseason = x_inseason[column_to_select].values

prediction_value = model.predict(x_inseason)


In [81]:

# ----------------------------------------------------
# Get the proba to get the Metric evaluation per game
# We will comapre proba to win from team 1 vs team 2
# It leads to have 1 lign per game
prediction_proba = model.predict_proba(x_inseason)
prediction_proba_df = pd.DataFrame(prediction_proba)
prediction_proba_df.columns = ["prediction_proba_df_0", "prediction_proba_df_1"]

# ---------------------------------------------

evaluation_accuracy = accuracy_score(y_inseason, prediction_value)


nba_games_inseasonn_w_pred = nba_games_inseason_dataset_final[
    [
        target_column,
        group_cv_variable,
        "id_season",
        "tm",
        "opp",
    ]
]

nba_games_inseasonn_w_pred["prediction_proba_df_0"] = prediction_proba_df[
    "prediction_proba_df_0"
].copy()
nba_games_inseasonn_w_pred["prediction_proba_df_1"] = prediction_proba_df[
    "prediction_proba_df_1"
].copy()
nba_games_inseasonn_w_pred["prediction_value"] = prediction_value

nba_games_inseasonn_w_pred.to_csv("./models/nba_games_inseasonn_w_pred.csv", index=False)

nba_games_inseasonn_w_pred = nba_games_inseasonn_w_pred.rename(
    columns={
        "prediction_proba_df_0": "prediction_proba_df_loose",
        "prediction_proba_df_1": "prediction_proba_df_win",
    }
)



/var/folders/3x/hm8hmk5d5v71tnm88n9rfb8c0000gn/T/ipykernel_5545/1859685952.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba_games_inseasonn_w_pred["prediction_proba_df_0"] = prediction_proba_df[
/var/folders/3x/hm8hmk5d5v71tnm88n9rfb8c0000gn/T/ipykernel_5545/1859685952.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba_games_inseasonn_w_pred["prediction_proba_df_1"] = prediction_proba_df[
/var/folders/3x/hm8hmk5d5v71tnm88n9rfb8c0000gn/T/ipykernel_5545/1859685952.py:30: SettingWithCopyWarning: 

In [82]:
nba_games_inseasonn_w_pred[nba_games_inseasonn_w_pred['tm']=='ATL']

,results,id,id_season,tm,opp,prediction_proba_df_loose,prediction_proba_df_win,prediction_value
0,1,2024-10-25_CHO_ATL,2025,ATL,CHO,0.371951,0.628049,1
1,0,2024-10-27_ATL_OKC,2025,ATL,OKC,0.638288,0.361712,0
2,0,2024-10-28_WAS_ATL,2025,ATL,WAS,0.547352,0.452648,0
3,0,2024-10-30_ATL_WAS,2025,ATL,WAS,0.515802,0.484198,0
4,0,2024-11-01_SAC_ATL,2025,ATL,SAC,0.517138,0.482862,0
5,1,2024-11-03_ATL_NOP,2025,ATL,NOP,0.608997,0.391003,0
6,0,2024-11-04_BOS_ATL,2025,ATL,BOS,0.716098,0.283902,0
7,1,2024-11-06_NYK_ATL,2025,ATL,NYK,0.639981,0.360019,0
8,0,2024-11-08_ATL_DET,2025,ATL,DET,0.613944,0.386056,0
9,0,2024-11-09_CHI_ATL,2025,ATL,CHI,0.504568,0.495432,0


In [68]:

opponent_features = [
    "id",
    "tm",
    "opp",
    "prediction_proba_df_loose",
    "prediction_proba_df_win",
]

nba_games_inseasonn_w_pred_opp = nba_games_inseasonn_w_pred[opponent_features]

nba_games_inseasonn_w_pred = pd.merge(
    nba_games_inseasonn_w_pred,
    nba_games_inseasonn_w_pred_opp,
    how="left",
    left_on=["id", "tm", "opp"],
    right_on=["id", "opp", "tm"],
)

nba_games_inseasonn_w_pred = rename_opponent_columns(nba_games_inseasonn_w_pred)

nba_games_inseasonn_w_pred["pred_results_1_line_game"] = np.where(
    nba_games_inseasonn_w_pred["prediction_proba_df_win"]
    > nba_games_inseasonn_w_pred["prediction_proba_df_win_opp"],
    1,
    0,
)

nba_games_inseasonn_w_pred = nba_games_inseasonn_w_pred.drop_duplicates(
    subset=["id"], 
    keep="first"
    )


In [69]:
nba_games_inseasonn_w_pred.head()

,results,id,id_season,tm,opp,prediction_proba_df_loose,prediction_proba_df_win,prediction_value,tm_opp,opp_opp,prediction_proba_df_loose_opp,prediction_proba_df_win_opp,pred_results_1_line_game
0,1,2024-10-25_CHO_ATL,2025,ATL,CHO,0.371951,0.628049,1,CHO,ATL,0.638362,0.361638,1
1,0,2024-10-27_ATL_OKC,2025,ATL,OKC,0.638288,0.361712,0,OKC,ATL,0.494925,0.505075,0
2,0,2024-10-28_WAS_ATL,2025,ATL,WAS,0.547352,0.452648,0,WAS,ATL,0.496614,0.503386,0
3,0,2024-10-30_ATL_WAS,2025,ATL,WAS,0.515802,0.484198,0,WAS,ATL,0.500987,0.499013,0
4,0,2024-11-01_SAC_ATL,2025,ATL,SAC,0.517138,0.482862,0,SAC,ATL,0.454982,0.545018,0


In [71]:
nba_games_inseasonn_w_pred = nba_games_inseasonn_w_pred[[
    'id', 'id_season', 'tm', 'opp', 
    'results', 'prediction_value', 'pred_results_1_line_game',
    'prediction_proba_df_loose',	'prediction_proba_df_win', 
    'prediction_proba_df_loose_opp',	'prediction_proba_df_win_opp']]

In [72]:
nba_games_inseasonn_w_pred.head()

,id,id_season,tm,opp,results,prediction_value,pred_results_1_line_game,prediction_proba_df_loose,prediction_proba_df_win,prediction_proba_df_loose_opp,prediction_proba_df_win_opp
0,2024-10-25_CHO_ATL,2025,ATL,CHO,1,1,1,0.371951,0.628049,0.638362,0.361638
1,2024-10-27_ATL_OKC,2025,ATL,OKC,0,0,0,0.638288,0.361712,0.494925,0.505075
2,2024-10-28_WAS_ATL,2025,ATL,WAS,0,0,0,0.547352,0.452648,0.496614,0.503386
3,2024-10-30_ATL_WAS,2025,ATL,WAS,0,0,0,0.515802,0.484198,0.500987,0.499013
4,2024-11-01_SAC_ATL,2025,ATL,SAC,0,0,0,0.517138,0.482862,0.454982,0.545018


In [73]:
nba_games_inseasonn_w_pred[nba_games_inseasonn_w_pred['id'].isin(['2024-10-25_CHO_ATL', '2024-10-25_ATL_CHO'])]

,id,id_season,tm,opp,results,prediction_value,pred_results_1_line_game,prediction_proba_df_loose,prediction_proba_df_win,prediction_proba_df_loose_opp,prediction_proba_df_win_opp
0,2024-10-25_CHO_ATL,2025,ATL,CHO,1,1,1,0.371951,0.628049,0.638362,0.361638
